In [1]:
import requests
import pandas as pd

In [2]:
coordinates = {"JFK": [40.6446,-73.7797],
               "ORD": [41.98043450764176, -87.91257800108377],
               "MCO": [28.426901098825073, -81.31249734360168],
               "SYR": [43.1113, -76.1143]}

weatherbit_historical_api_endpoint = "https://api.weatherbit.io/v2.0/history/hourly"

# Update location in params for weather data of different airport
loc = "JFK"
params = {
    "lat": coordinates[loc][0],
    "lon": coordinates[loc][1],
    "tz": "local",
    "key": "82559218ca6d48d9892cd1b829d10263"
}

features = ['datetime', 'timestamp_local', 'temp', 'app_temp', 'clouds', 'precip', 'rh', 
            'wind_spd', 'wind_gust_spd', 'wind_dir', 'weather', 'pres', 'slp', 'vis', 'snow']

weather_df = pd.DataFrame()

In [ ]:
# We can only get maximum of 2 years of data from a single request.
# So, repeat this cell for the years: 2019 - 2020, 2021 - 2023
# Note: 1 key can get us a maximum of 4 years of data.

params["start_date"] = "2019-01-01"
params["end_date"] = "2020-12-31"

response = requests.get(weatherbit_historical_api_endpoint, params = params)
res = response.json()
data = [
        {
            feat: row['weather']['description'] if feat == 'weather' else row.get(feat) for feat in features
            } for row in res['data']
        ]
weather_df = pd.concat(weather_df, pd.DataFrame(data), ignore_index = True)

weather_df.to_csv(f'data/weather/weather_{loc}_2019_2020.csv', index = False)

## Forecast data: April 19 - April 23, 2024

In [11]:
weatherbit_forecast_api_endpoint = "https://api.weatherbit.io/v2.0/forecast/hourly"

# Update the location in params
loc = "MCO"
params = {
    "lat": coordinates[loc][0],
    "lon": coordinates[loc][1],
    "key": "b843dfc4409d4598a42ec00ffd2fc5ec",
    "start_date": "2024-04-19",
    "end_date": "2024-04-23"
}

response = requests.get(weatherbit_forecast_api_endpoint, params = params)
res = response.json()
data = [
        {
            feat: row['weather']['description'] if feat == 'weather' else row.get(feat) for feat in features
            } for row in res['data']
        ]
weather_forecast_df = pd.DataFrame(data)
weather_forecast_df.to_csv(f'data/weather/weather_forecast_data_{loc.lower()}.csv', index = False)

In [12]:
df = pd.DataFrame() # merge forecasts

for file in ["data/weather/weather_forecast_data_jfk.csv",  "data/weather/weather_forecast_data_ord.csv",  "data/weather/weather_forecast_data_mco.csv"]:
    df = pd.concat([df, pd.read_csv(file)])

df.reset_index(inplace=True, drop=True)

df.to_csv("weather_forecast_data.csv", index=False)